## Introduction to PyTorch

In [ ]:
## Standard libraries
import os
import math
import numpy as np
import time
import pandas as pd

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
from matplotlib.colors import to_rgba
import seaborn as sns
sns.set()

## Progress bar
from tqdm.notebook import tqdm

In [ ]:
import torch
print("Using torch", torch.__version__)

In [ ]:
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

In [ ]:
if torch.cuda.is_available():
    print("Number of CUDA devices:", torch.cuda.device_count())
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
if torch.cuda.is_available():
    print("Device name:", torch.cuda.get_device_name(0))  # 0 for the first device

In [ ]:
if torch.cuda.is_available():
    properties = torch.cuda.get_device_properties(0)
    print("Device name:", properties.name)
    print("Total memory:", properties.total_memory//1024//1024//1024)

In [ ]:
if torch.cuda.is_available():
    print("Current device:", torch.cuda.current_device())

In [ ]:
torch.cuda.set_device(3) ## CHANGE DEVICE

In [ ]:
if torch.cuda.is_available():
    print("Current device:", torch.cuda.current_device())

### Tensors

Tensors are the PyTorch equivalent to Numpy arrays, with the addition to also have support for GPU acceleration (more on that later).
The name "tensor" is a generalization of concepts you already know. For instance, a vector is a 1-D tensor, and a matrix a 2-D tensor. When working with neural networks, we will use tensors of various shapes and number of dimensions.

Most common functions you know from numpy can be used on tensors as well. Actually, since numpy arrays are so similar to tensors, we can convert most tensors to numpy arrays (and back) but we don't need it too often.

In [ ]:
### Working with tensors, similar to tensorflow
x = torch.Tensor(2, 3, 4)
print(x)

The function `torch.Tensor` allocates memory for the desired tensor, but reuses any values that have already been in the memory. To directly assign values to the tensor during initialization, there are many alternatives including:

* `torch.zeros`: Creates a tensor filled with zeros
* `torch.ones`: Creates a tensor filled with ones
* `torch.rand`: Creates a tensor with random values uniformly sampled between 0 and 1
* `torch.randn`: Creates a tensor with random values sampled from a normal distribution with mean 0 and variance 1
* `torch.arange`: Creates a tensor containing the values $N,N+1,N+2,...,M$
* `torch.Tensor` (input list): Creates a tensor from the list elements you provide

#### Tensor to Numpy, and Numpy to Tensor

Tensors can be converted to numpy arrays, and numpy arrays back to tensors. To transform a numpy array into a tensor, we can use the function `torch.from_numpy`:

In [ ]:
##
np_arr = np.array([[1, 2], [3, 4]])
tensor = torch.from_numpy(np_arr)

print("Numpy array:", np_arr)
print("PyTorch tensor:", tensor)

To transform a PyTorch tensor back to a numpy array, we can use the function `.numpy()` on tensors:

In [ ]:
tensor = torch.arange(4)
np_arr = tensor.numpy()

print("PyTorch tensor:", tensor)
print("Numpy array:", np_arr)


The conversion of tensors to numpy require the tensor to be on the CPU, and not the GPU . In case you have a tensor on GPU, you need to call `.cpu()` on the tensor beforehand. Hence, you get a line like `np_arr = tensor.cpu().numpy()`.

In [ ]:
## Move to GPU device
tensor.to(device)

#### Operations

Most operations that exist in numpy, also exist in PyTorch. A full list of operations can be found in the [PyTorch documentation](https://pytorch.org/docs/stable/tensors.html#), but we will review the most important ones here.

The simplest operation is to add two tensors:

In [ ]:
x1 = torch.rand(2, 3)
x2 = torch.rand(2, 3)
y = x1 + x2

print("X1", x1)
print("X2", x2)
print("Y", y)

Calling `x1 + x2` creates a new tensor containing the sum of the two inputs. However, we can also use in-place operations that are applied directly on the memory of a tensor. We therefore change the values of `x2` without the chance to re-accessing the values of `x2` before the operation. An example is shown below:

In [ ]:
x1 = torch.rand(2, 3)
x2 = torch.rand(2, 3)
print("X1 (before)", x1)
print("X2 (before)", x2)

x2.add_(x1)
print("X1 (after)", x1)
print("X2 (after)", x2)

In [ ]:
## comparison CPU vs GPU

In [ ]:
x = torch.randn(5000, 5000)

## CPU version
start_time = time.time()
_ = torch.matmul(x, x)
end_time = time.time()
print(f"CPU time: {(end_time - start_time):6.5f}s")

## GPU version
x = x.to(device)
_ = torch.matmul(x, x)  # First operation to 'burn in' GPU
# CUDA is asynchronous, so we need to use different timing functions
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
start.record()
_ = torch.matmul(x, x)
end.record()
torch.cuda.synchronize()  # Waits for everything to finish running on the GPU
print(f"GPU time: {0.001 * start.elapsed_time(end):6.5f}s")  # Milliseconds to seconds

### Dynamic Computation Graph and Backpropagation

One of the main reasons for using PyTorch in Deep Learning projects is that we can automatically get **gradients/derivatives** of functions that we define. We will mainly use PyTorch for implementing neural networks, and they are just fancy functions. If we use weight matrices in our function that we want to learn, then those are called the **parameters** or simply the **weights**.

If our neural network would output a single scalar value, we would talk about taking the **derivative**, but you will see that quite often we will have **multiple** output variables ("values"); in that case we talk about **gradients**. It's a more general term.

Given an input $\mathbf{x}$, we define our function by **manipulating** that input, usually by matrix-multiplications with weight matrices and additions with so-called bias vectors. As we manipulate our input, we are automatically creating a **computational graph**. This graph shows how to arrive at our output from our input. 
PyTorch is a **define-by-run** framework; this means that we can just do our manipulations, and PyTorch will keep track of that graph for us. Thus, we create a dynamic computation graph along the way.

So, to recap: the only thing we have to do is to compute the **output**, and then we can ask PyTorch to automatically get the **gradients**. 

> **Note:  Why do we want gradients?** Consider that we have defined a function, a neural net, that is supposed to compute a certain output $y$ for an input vector $\mathbf{x}$. We then define an **error measure** that tells us how wrong our network is; how bad it is in predicting output $y$ from input $\mathbf{x}$. Based on this error measure, we can use the gradients to **update** the weights $\mathbf{W}$ that were responsible for the output, so that the next time we present input $\mathbf{x}$ to our network, the output will be closer to what we want.

The first thing we have to do is to specify which tensors require gradients. By default, when we create a tensor, it does not require gradients.

In [ ]:
x = torch.ones((3,))
print(x.requires_grad)

We can change this for an existing tensor using the function `requires_grad_()` (underscore indicating that this is a in-place operation). Alternatively, when creating a tensor, you can pass the argument `requires_grad=True` to most initializers we have seen above.

In [ ]:
x.requires_grad_(True)
print(x.requires_grad)

## Example

In [ ]:
x = torch.arange(3, dtype=torch.float32, requires_grad=True) # Only float tensors can have gradients
print("X", x)

In order to get familiar with the concept of a computation graph, we will create one for the following function:

$$y = \frac{1}{\ell(x)}\sum_i \left[(x_i + 2)^2 + 3\right],$$

where we use $\ell(x)$ to denote the number of elements in $x$. In other words, we are taking a mean here over the operation within the sum. You could imagine that $x$ are our parameters, and we want to optimize (either maximize or minimize) the output $y$. For this, we want to obtain the gradients $\partial y / \partial \mathbf{x}$. For our example, we'll use $\mathbf{x}=[0,1,2]$ as our input.

In [ ]:
a = x + 2
b = a ** 2
c = b + 3
y = c.mean()
print("Y", y)

We can perform backpropagation on the computation graph by calling the function `backward()` on the last output, which effectively calculates the gradients for each tensor that has the property `requires_grad=True`:

In [ ]:
y.backward()

`x.grad` will now contain the gradient $\partial y/ \partial \mathcal{x}$, and this gradient indicates how a change in $\mathbf{x}$ will affect output $y$ given the current input $\mathbf{x}=[0,1,2]$:

In [ ]:
print(x.grad)

## Data Example, in comparison with Tensorflow

In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight','Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names,na_values='?', comment='\t',sep=' ', skipinitialspace=True)

In [ ]:

dataset = raw_dataset.copy()
dataset.tail()


In [ ]:
dataset.isna().sum()

In [ ]:
dataset = dataset.dropna()

In [ ]:
dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})

In [ ]:
dataset = pd.get_dummies(dataset, columns=['Origin'], prefix='', prefix_sep='')
dataset.tail()

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
sns.pairplot(train_dataset[['MPG', 'Cylinders', 'Displacement', 'Weight']], diag_kind='kde')

In [ ]:
## Import torch utilities
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

The package `torch.nn` defines a series of useful classes like linear networks layers, activation functions, loss functions etc. A full list can be found [here](https://pytorch.org/docs/stable/nn.html). In case you need a certain network layer, check the documentation of the package first before writing the layer yourself as the package likely contains the code for it already. We import it below:

In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()
#train_features = train_dataset.copy()
#test_features = test_dataset.copy()
train_labels = train_features.pop('MPG')
test_labels = test_features.pop('MPG')
#train_features = train_features['Horsepower']
#test_features = test_features['Horsepower']

In [ ]:
# Normalize the inputs
class Normalize:
    def __init__(self, features):
        features = torch.tensor(features, dtype=torch.float32)  # Convert to tensor
        self.mean = torch.mean(features, dim=0)
        self.std = torch.std(features, dim=0)

    def __call__(self, features):
        features = torch.tensor(features, dtype=torch.float32)  # Convert to tensor
        return (features - self.mean) / self.std

In [ ]:

X_train = train_features.values.astype(float)
y_train = train_labels.values.astype(float)
X_test = test_features.values.astype(float)
y_test = test_labels.values.astype(float)
##
#X_train = X_train.reshape(-1,1)
#X_test = X_test.reshape(-1,1)

In [ ]:
normalizer = Normalize(X_train)

In [ ]:
print(normalizer.mean)

In [ ]:
#X_train_normalized = normalizer(X_train)
#X_test_normalized = normalizer(X_test)

In [ ]:
# Define a custom dataset
class CarDataset(data.Dataset):
    def __init__(self, features, targets, normalizer):
        self.features = normalizer(features)
        self.targets = torch.tensor(targets, dtype=torch.float32)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [ ]:
# Create dataset and DataLoader
dataset_train = CarDataset(X_train, y_train, normalizer)
dataloader_train = data.DataLoader(dataset_train, batch_size=8, shuffle=True, drop_last=False)

# Example usage
#for batch_features, batch_targets in dataloader:
#    print("Batch features:", batch_features)
#    print("Batch targets:", batch_targets)

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim1):
        super(SimpleNN, self).__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim1)   # Dense layer with hidden_dim units
        self.act_fn = nn.ReLU() ## Activation
        self.output = nn.Linear(hidden_dim1, 1)# Output layer

    def forward(self, x):
        x = self.linear1(x) 
        x = self.act_fn(x) # Apply activation
        x = self.output(x)                     # Output layer (no activation for regression)
        return x

In [ ]:
model = SimpleNN(X_train.shape[1],40)
print(model)
model.to(device)
#push model to device

In [ ]:
for name, param in model.named_parameters():
    print(f"Parameter {name}, shape {param.shape}")

In [ ]:
# Calculate total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params}")

In [ ]:
loss_module = nn.L1Loss() ## Mean Absolute Error (nn.MSELoss --> Mean squared error
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
logging_dir = 'runs/experiments/pt'
writer = SummaryWriter(logging_dir)
model_plotted = False
model.train()
# Train the model
epochs = 100
error = []
for epoch in tqdm(range(epochs)):
    epoch_loss = 0.0
    for batch_features, batch_targets in dataloader_train:
        # push data to device
        batch_features = batch_features.to(device)
        batch_targets = batch_targets.to(device)
        # For the very first batch, we visualize the computation graph in TensorBoard
        if not model_plotted:
            writer.add_graph(model, batch_features)
            model_plotted = True
        # Forward pass
        predictions = model(batch_features)
        loss = loss_module(predictions.squeeze(), batch_targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

      # Add average loss to TensorBoard
    epoch_loss /= len(dataloader_train)
    writer.add_scalar('training_loss',
                      epoch_loss,
                      global_step = epoch + 1)


    writer.close()
    error.append(epoch_loss)

In [ ]:
plt.plot(error, label='loss')
plt.ylim([0, 10])
plt.xlabel('Epoch')
plt.ylabel('Error [MPG]')
plt.legend()
plt.grid(True)

In [ ]:
with torch.no_grad():
    xx = torch.from_numpy(np.linspace(0.0, 250, 251).reshape(-1,1))
    xx = xx.to(torch.float32)
    xx = normalizer(xx)
    xx = xx.to(device)
    model.eval()
    yy = model(xx).squeeze()
    yf = yy.cpu().numpy()

In [ ]:
try:
    xf = np.linspace(0.0, 250, 251)
    plt.scatter(train_features, train_labels, label='Data')
    plt.plot(xf, yf, color='k', label='Predictions')
    plt.xlabel('Horsepower')
    plt.ylabel('MPG')
    plt.xlim(0,250)
    plt.ylim(0,50)
    plt.legend()
except:
    pass

In [ ]:
#state_dict = model.state_dict()
#print(state_dict)

In [ ]:
#torch.save(state_dict, "our_model.tar")

In [ ]:
# Load state dict from the disk (make sure it is the same name as above)
#state_dict = torch.load("our_model.tar")

# Create a new model and load the state
#new_model = SimpleNN(X_train.shape[1], 32)
#new_model.load_state_dict(state_dict)

In [ ]:
#print(new_model.state_dict())

In [ ]:
# Create dataset and DataLoader for test set
dataset_test = CarDataset(X_test, y_test, normalizer)
dataloader_test = data.DataLoader(dataset_test, batch_size=100, shuffle=False, drop_last=False)

In [ ]:
# Example inference
model.eval()
with torch.no_grad():
    for batch_features, batch_targets in dataloader_test:
        batch_features = batch_features.to(device)
        batch_targets = batch_targets.to(device)
        preds = model(batch_features)
        preds = preds.squeeze()
        loss = loss_module(preds, batch_targets)
print('Model MAE (loss) = {:.4}'.format(loss))

In [ ]:
predictions =  preds.cpu().numpy()

In [ ]:
a = plt.axes(aspect='equal')
plt.scatter(y_test, predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)

In [ ]:
error = predictions - y_test
plt.hist(error, bins=25)
plt.xlabel('Prediction Error [MPG]')
_ = plt.ylabel('Count')

### Exercise
TODO: Repeat the analysis for the same architecture as shown in the `tf_example.ipynb` for the `Horsepower` only and all 9 variable

## References
https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial2/Introduction_to_PyTorch.html
https://pytorch.org/docs/stable/nn.html

In [ ]:
#%load_ext tensorboard 

In [ ]:
#%tensorboard --logdir runs/experiments --port 6008